In [ ]:
from TDLR.dissolve import dissolve
from TDLR.timedependentlocations_AFVG import timedependentlocations_AFVG
from TDLR.timedependentlocations_PG import timedependentlocations_PG
from TDLR.timedependentlocations_RBG import timedependentlocations_RBG
from TDLR.timedependentrelations import timedependentrelations

In [ ]:
# path to settings file
settings_file = r"D:\jupyter notebooks\4019.10 TimeDependentLocatios HDSR\final\TDLR_settings.json"

In [ ]:
# # dissolve geodump shapefiles
# dissolve(settings_file)

# derive time dependency afvoergebieden
timedependentlocations_AFVG(settings_file)

# derive time dependency peilgebieden
timedependentlocations_PG(settings_file)

# derive time dependency rioolbemalingsgebieden
timedependentlocations_RBG(settings_file)

# derive time dependent relations
timedependentrelations(settings_file)

In [13]:
import pandas as pd

In [60]:
fp_log_old = r"D:\jupyter notebooks\4019.10 TimeDependentLocatios HDSR\from_Joost\Stap 2 - Time Dependent Locations\Log_file_PG_base_20190301.csv"
fp_log_new = r"D:\jupyter notebooks\4019.10 TimeDependentLocatios HDSR\from_Joost\Stap 2 - Time Dependent Locations\Log_file_PG_base_20190401.csv"

In [61]:
df_diff_log = get_df_diff_log(fp_log_new, fp_log_old)

In [70]:
chart = get_chart_diff_log(df_diff_log)

In [65]:
import hkvportal as hp

In [66]:
ds = 'https://tsws.hkvservices.nl/mangrove.ws/'
uid = '236ebLIeCDdnKbfGe956IvZetIv1'
dp = hp.Service(ds, uid=uid)

Dataservice is recognized. https://tsws.hkvservices.nl/mangrove.ws/entry.asmx?WSDL will be used as portal


In [67]:
dp.create_database('tdlr_hdsr')

{'database': 'tdlr_hdsr', 'version': '0.3.0.0', 'date': '2019-12-19 12:45:22'}

In [72]:
dp.new_entry('tdlr_hdsr', 'log_dif_1904_1903', chart.to_html())

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'tdlr_hdsr',key:'log_dif_1904_1903'}&contentType=SET_BY_USER


In [63]:
import pandas as pd


def get_df_diff_log(fp_log_new, fp_log_old):

    # read log entries as dataframe
    df_log_old = pd.read_csv(fp_log_old, index_col=0)
    df_log_new = pd.read_csv(fp_log_new, index_col=0)

    # select dates that exist in both log files
    exist_both = list(set(df_log_new.Date_base).intersection(df_log_old.Date_base))

    # select the rows in both files maching the interesecting dump dates
    df_log_old_sel = (
        df_log_old[df_log_old["Date_base"].isin(exist_both)]
        .sort_values(by=["Date_base"], ascending=False)
        .reset_index(drop=True)
    )
    df_log_new_sel = (
        df_log_new[df_log_new["Date_base"].isin(exist_both)]
        .sort_values(by=["Date_base"], ascending=False)
        .reset_index(drop=True)
    )

    if not df_log_new_sel.shape[0] == df_log_old_sel.shape[0]:
        print("DataFrames dont match, return")

    # compute differences between the two log files
    df_diff_log = (
        df_log_new_sel.loc[:, df_log_new_sel.columns != "Date_base"]
        - df_log_old_sel.loc[:, df_log_old_sel.columns != "Date_base"]
    )

    # insert the date column
    df_diff_log.insert(0, "Date_base", df_log_new_sel.Date_base)

    return df_diff_log


def get_chart_diff_log(df_diff_log):
    import altair as alt

    df_diff_log.Date_base = pd.to_datetime(df_diff_log.Date_base, format="%Y_%m%d")

    columns = list(df_diff_log.columns[1::])
    nearest = alt.selection(
        encodings=["x"], type="single", nearest=True, on="mouseover", empty="none"
    )

    lines = (
        alt.Chart(df_diff_log)
        .transform_fold(columns, as_=["logging class", "no. occurences"])
        .mark_line()
        .encode(x="yearmonth(Date_base)", color="logging class:N", y="no. occurences:Q")
    )

    # Draw points on the line, and highlight based on selection
    points = lines.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw a rule at the location of the selection
    rules = (
        alt.Chart(df_diff_log)
        .mark_rule(color="gray")
        .encode(
            x="yearmonth(Date_base)",
            tooltip=["{}:Q".format(col) for col in columns] + ["yearmonth(Date_base)"],
            opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        )
        .add_selection(nearest)
    )

    chart = alt.layer(lines, points, rules).properties(width=600, height=250)
    return chart

In [ ]:
# # script to compare the loggings of two runs
# import pandas as pd

# log_201904 = pd.read_csv('Log_file_PG_base_20190401.csv',index_col = 0)
# log_201904_vanaf03 = log_201904[log_201904.index > 0].reset_index(drop = True)

# log_201903 = pd.read_csv('Log_file_PG_base_20190301.csv',index_col = 0)

# logg_diff = log_201904_vanaf03.copy()


# for col in log_201904_vanaf03.columns:
#     if col == 'Date_base':
#         continue
#     logg_diff[col] = log_201904_vanaf03[col] - log_201903[col]

# logg_diff.to_csv('Log_diff_201904_2019_03.csv',index = False)